In [7]:
from ase import Atoms
from ase.build import fcc100, add_vacuum
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.data import atomic_numbers
from ase.ga.startgenerator import StartGenerator
from ase.ga.utilities import closest_distances_generator
from ase.ga.utilities import get_all_atom_types
from ase.io import read
from ase.optimize.sciopt import SciPyFminCG
from ase.visualize import view

import numpy as np


In [8]:
surface_size = (6, 6, 3)
slab = fcc100('Au', surface_size, periodic=True)
add_vacuum(slab, 10)
slab.set_constraint(FixAtoms(mask=len(slab) * [True]))

# define the volume in which the adsorbed cluster is optimized
# the volume is defined by a corner position (p0)
# and three spanning vectors (v1, v2, v3)
pos = slab.get_positions()
cell = slab.get_cell()
print(cell[0,0])
p0 = np.array([cell[0, 0]/10*3, cell[1, 1]/10*3, max(pos[:, 2]) + 2.])
v1 = cell[0, :] * 0.4
v2 = cell[1, :] * 0.4
v3 = cell[2, :] * .3

# Define the composition of the atoms to optimize
atom_numbers = 12 * [atomic_numbers['Pt']]

# define the closest distance two atoms of a given species can be to each other
unique_atom_types = get_all_atom_types(slab, atom_numbers)
blmin = closest_distances_generator(atom_numbers=unique_atom_types,
                                    ratio_of_covalent_radii=0.7)

# create the starting population
sg = StartGenerator(slab, atom_numbers, blmin,
                    box_to_place_in=[p0, [v1, v2, v3]])
# view(slab)


17.309974003446687


In [9]:
atoms = sg.get_new_candidate()
atoms.set_calculator(EMT())
traj_name = 'images/pt12_au100_emt.traj'
dyn = SciPyFminCG(atoms, logfile='-', trajectory=traj_name)
dyn.run(fmax=0.25)

traj = read(traj_name, index=':')
view(traj, viewer='ngl')


             Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
SciPyFminCG:    0 23:57:51       49.363261*      19.7593
SciPyFminCG:    1 23:57:51       41.496118*       5.4398
SciPyFminCG:    2 23:57:51       40.374631*       3.9238
SciPyFminCG:    3 23:57:51       39.885964*       2.6866
SciPyFminCG:    4 23:57:51       39.254358*       2.1988
SciPyFminCG:    5 23:57:52       38.988242*       2.5141
SciPyFminCG:    6 23:57:52       38.705617*       2.0617
SciPyFminCG:    7 23:57:52       38.120604*       2.7156
SciPyFminCG:    8 23:57:53       37.715791*       1.7624
SciPyFminCG:    9 23:57:53       37.383131*       1.2369
SciPyFminCG:   10 23:57:53       37.173343*       1.1484
SciPyFminCG:   11 23:57:53       36.683269*       1.6972
SciPyFminCG:   12 23:57:54       36.538500*       1.0284
SciPyFminCG:   13 23:57:54       36.369510*       0.7592
SciPyFminCG:   14 23:57:54       36.258603*       0.9085
SciPyFminCG:   15 23:57:54       36.1847